# Radbert

### Data Loading and Preprocessing

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import classification_report, roc_auc_score
import random
import torch.nn as nn
import chardet

# Define 14 labels
label_columns = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
    'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
    'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'
]

# --- Data Loading and Preprocessing ---
study_list_path = 'data/cxr-study-list.csv'
labels_path = 'data/mimic-cxr-2.0.0-chexpert.csv'

df_studies = pd.read_csv(study_list_path)
df_labels = pd.read_csv(labels_path)

merged_data = pd.merge(df_studies, df_labels, on=['subject_id', 'study_id'])

# Data cleaning
merged_data[label_columns] = merged_data[label_columns].fillna(0)
for col in label_columns:
    if col != 'No Finding':
        merged_data[col] = merged_data.apply(
            lambda row: 0 if row['No Finding'] == 1 else row[col], axis=1
        )
merged_data[label_columns] = merged_data[label_columns].apply(pd.to_numeric, errors='coerce')
# Replace -1 with 0 in labels
merged_data[label_columns] = merged_data[label_columns].replace(-1, 0)

# Sample 30,000 records before loading reports
merged_data_sampled = merged_data.sample(n=30000, random_state=42)
print(f"Sampled {len(merged_data_sampled)} records.")

# Load report text with progress bar and encoding detection
def load_report(subject_id, study_id):
    subject_id_str = str(int(subject_id))
    study_id_str = str(int(study_id))
    report_file_path = f'data/files/p{subject_id_str[:2]}/p{subject_id_str}/s{study_id_str}.txt'
    try:
        with open(report_file_path, 'rb') as f:
            raw_data = f.read()
            encoding = chardet.detect(raw_data)['encoding']
            text = raw_data.decode(encoding)
            # Remove duplicate lines
            lines = text.split('\n')
            unique_lines = list(dict.fromkeys(lines))  # Maintain order
            return '\n'.join(unique_lines)
    except FileNotFoundError:
        return None
    except Exception as e:
        print(f"Error loading report for subject {subject_id}, study {study_id}: {e}")
        return None

merged_data_sampled['report'] = merged_data_sampled.apply(lambda row: load_report(row['subject_id'], row['study_id']), axis=1)
merged_data_sampled = merged_data_sampled.dropna(subset=['report'])

print("Report loading completed.")

# Split data into train and temp (50% train, 50% temp)
train_data, temp_data = train_test_split(merged_data_sampled, test_size=0.5, random_state=42)

# Split temp into validation and test (60% validation, 40% test of temp)
val_data, test_data = train_test_split(temp_data, test_size=0.4, random_state=42)

# Calculate the actual proportions
total_samples = len(merged_data_sampled)
train_prop = len(train_data) / total_samples
val_prop = len(val_data) / total_samples
test_prop = len(test_data) / total_samples

print(f"Train proportion: {train_prop:.2f}, Validation proportion: {val_prop:.2f}, Test proportion: {test_prop:.2f}")

# --- Dataset Definition ---
class RadiologyDataset(Dataset):
    def __init__(self, reports, labels, tokenizer, max_length=512):
        self.reports = reports
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reports)

    def __getitem__(self, idx):
        report = self.reports[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(report, truncation=True, padding='max_length', 
                                 max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

print("Dataset definition completed.")

Sampled 30000 records.
Report loading completed.
Train proportion: 0.50, Validation proportion: 0.30, Test proportion: 0.20
Dataset definition completed.


### Use StanfordAIMI RadBERT model

In [ ]:
# --- Model Configuration ---
model_name = "StanfordAIMI/RadBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForMaskedLM.from_pretrained(model_name)

# Define a custom model with a classification head
class CustomBERTModel(nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomBERTModel, self).__init__()
        self.encoder = base_model.bert  # Assuming RadBERT is BertForMaskedLM
        self.classifier = nn.Linear(self.encoder.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token representation
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)
        return (loss, logits) if loss is not None else logits

model = CustomBERTModel(base_model, num_labels=len(label_columns))

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Using device: {device}")

# Prepare datasets and dataloaders
train_dataset = RadiologyDataset(train_data['report'].tolist(), train_data[label_columns].values, tokenizer)
val_dataset = RadiologyDataset(val_data['report'].tolist(), val_data[label_columns].values, tokenizer)
test_dataset = RadiologyDataset(test_data['report'].tolist(), test_data[label_columns].values, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
print("Data preparation and model configuration completed.")

# --- Model Training ---
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        loss, _ = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Training Loss: {avg_train_loss:.4f}")

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            loss, _ = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Validation Loss: {avg_val_loss:.4f}")

# --- Save Model ---
torch.save(model.state_dict(), 'aimi_rad_bert_model.pth')
tokenizer.save_pretrained('aimi_rad_bert_model')
print("Training completed and model saved.")

Some weights of the model checkpoint at StanfordAIMI/RadBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Data preparation and model configuration completed.


Training Epoch 1/3:  17%|█▋        | 324/1875 [02:16<10:53,  2.37it/s]

### Evaluate on Test Set

In [ ]:
# --- Evaluate on Test Set ---
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs  # Directly use outputs as logits
            preds = torch.sigmoid(logits).cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    # Calculate ROC AUC scores for each label
    roc_auc_scores = {}
    for i, label in enumerate(label_columns):
        try:
            roc_auc = roc_auc_score(all_labels[:, i], all_preds[:, i])
            roc_auc_scores[label] = roc_auc
        except ValueError as e:
            print(f"Warning: ROC AUC could not be calculated for {label}. Error: {e}")
            roc_auc_scores[label] = np.nan
    
    # Print ROC AUC scores
    print("\nROC AUC Scores:")
    for label, score in roc_auc_scores.items():
        if not np.isnan(score):
            print(f"{label}: {score:.4f}")
        else:
            print(f"{label}: N/A")
    
    # Convert predictions to binary values using 0.5 threshold
    binary_preds = (all_preds > 0.5).astype(int)
    
    # Generate classification report for each label
    print("\nClassification Report:")
    for i, label in enumerate(label_columns):
        print(f"Label: {label}")
        print(classification_report(all_labels[:, i], binary_preds[:, i], target_names=['Negative', 'Positive']))
    
    # Calculate macro average ROC AUC (excluding NaN values)
    valid_scores = [score for score in roc_auc_scores.values() if not np.isnan(score)]
    if valid_scores:
        macro_roc_auc = np.mean(valid_scores)
        print(f"\nMacro Average ROC AUC: {macro_roc_auc:.4f}")
    else:
        print("\nMacro Average ROC AUC: N/A")

# Evaluate on test set
evaluate_model(model, test_loader, device)

Evaluating: 100%|██████████| 750/750 [01:31<00:00,  8.18it/s]



ROC AUC Scores:
Atelectasis: 0.9641
Cardiomegaly: 0.9847
Consolidation: 0.9538
Edema: 0.9688
Enlarged Cardiomediastinum: 0.6739
Fracture: 0.9472
Lung Lesion: 0.9736
Lung Opacity: 0.9925
No Finding: 0.9932
Pleural Effusion: 0.9838
Pleural Other: 0.9666
Pneumonia: 0.8358
Pneumothorax: 0.9802
Support Devices: 0.9892

Classification Report:
Label: Atelectasis
              precision    recall  f1-score   support

    Negative       0.96      1.00      0.98      4793
    Positive       0.98      0.85      0.91      1207

    accuracy                           0.97      6000
   macro avg       0.97      0.92      0.95      6000
weighted avg       0.97      0.97      0.97      6000

Label: Cardiomegaly
              precision    recall  f1-score   support

    Negative       0.98      0.99      0.99      4769
    Positive       0.95      0.94      0.94      1231

    accuracy                           0.98      6000
   macro avg       0.97      0.96      0.96      6000
weighted avg       0.9

In [ ]:
# --- Show Random Test Prediction ---
def show_random_test_prediction(test_data, label_columns, tokenizer, model, device):
    # Randomly select one sample from the test dataset
    random_idx = random.randint(0, len(test_data) - 1)
    sample_report = test_data.iloc[random_idx]['report']  # Report text
    sample_labels = test_data.iloc[random_idx][label_columns].values  # True labels

    # Prepare model input
    encoding = tokenizer(
        sample_report,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'  # Return PyTorch tensors
    )
    input_ids = encoding['input_ids'].to(device)  # Input IDs
    attention_mask = encoding['attention_mask'].to(device)  # Attention mask

    # Model prediction
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs  # Directly use outputs as logits
        preds = torch.sigmoid(logits).cpu().numpy()[0]  # Convert to probabilities

    # Process prediction results
    binary_preds = (preds > 0.5).astype(int)  # Probabilities greater than 0.5 are 1, otherwise 0

    # Display results
    print("Selected Report:")
    print(sample_report)
    print("\nTrue Labels:")
    for label, value in zip(label_columns, sample_labels):
        print(f"{label}: {value}")
    print("\nPredicted Probabilities (Raw):")
    for label, prob in zip(label_columns, preds):
        print(f"{label}: {prob:.4f}")
    print("\nPredicted Binary Labels:")
    for label, pred in zip(label_columns, binary_preds):
        print(f"{label}: {pred}")

# Call the function
show_random_test_prediction(test_data, label_columns, tokenizer, model, device)

Selected Report:
                                 FINAL REPORT
 SINGLE FRONTAL VIEW OF THE CHEST
 
 REASON FOR EXAM:  Respiratory failure.
 
 Comparison is made with prior study ___.
 
 Cardiomegaly and widened mediastinum are unchanged.  NG tube tip is in the
 lower hemithorax midline, unchanged.  Left IJ catheter tip is in the mid SVC. 
 Cardiomediastinal contours are unchanged.  Large bilateral effusions
 associated with adjacent atelectasis and multifocal opacities, worse in the
 lower lobes, right greater than left and in the right apex are grossly
 unchanged.  There are no new lung abnormalities or pneumothorax.


True Labels:
Atelectasis: 1.0
Cardiomegaly: 1.0
Consolidation: 0.0
Edema: 0.0
Enlarged Cardiomediastinum: -1.0
Fracture: 0.0
Lung Lesion: 0.0
Lung Opacity: 1.0
No Finding: 0.0
Pleural Effusion: 1.0
Pleural Other: 0.0
Pneumonia: 0.0
Pneumothorax: -1.0
Support Devices: 1.0

Predicted Probabilities (Raw):
Atelectasis: 0.7502
Cardiomegaly: 0.1656
Consolidation: 0.0168
Edema